In [19]:
import hopsworks
from hsfs.client.exceptions import RestAPIError

# Establish connection to Hopsworks
project = hopsworks.login(
    api_key_value="dYTVrTVvbj6Qw82i.YGKHdS9snQYFgOADJIvLdvZ2n2S5BxIAOtvPUEmAyd56bvaG6xhhGyNM3nYbexaP"
)

# Access the Feature Store
fs = project.get_feature_store()

# List of feature groups and their versions
feature_groups = [
    {"name": "city_weather_features", "version": 1},
    {"name": "drivers_features", "version": 1},
    {"name": "routes_features", "version": 1},
    {"name": "routes_weather_features", "version": 1},
    {"name": "traffic_table_features", "version": 1},  # Corrected typo
    {"name": "truck_schedule_features", "version": 1},
    {"name": "trucks_features", "version": 1},
]
# Initialize an empty dictionary to hold DataFrames for each feature group
feature_group_data = {}

# Loop over the list of feature groups and download them
for fg in feature_groups:
    try:
        # Retrieve the feature group by its name and version
        feature_group = fs.get_feature_group(fg['name'], version=fg['version'])
        df = feature_group.read()  # Read the feature group as a DataFrame
        
        # Save the DataFrame in the dictionary with the feature group's name as the key
        feature_group_data[fg['name']] = df

        print(f"Downloaded feature group: {fg['name']} (version {fg['version']})")
        print(df.head())  # Optionally display the first few rows of the DataFrame

    except RestAPIError as e:
        print(f"Error downloading feature group: {fg['name']} (version {fg['version']})")
        print(e)

# After the loop, you will have each feature group stored in the `feature_group_data` dictionary
# Access each DataFrame like this: feature_group_data['city_weather'], etc.



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1044630
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.08s) 
Downloaded feature group: city_weather_features (version 1)
   index     city_id  temp  wind_speed    description  precip  humidity  \
0  33470  C-2c9e75ef    48           7         Cloudy     0.0        82   
1  52207  C-841ebdcb    48           7          Sunny     0.0        78   
2   3667  C-4fe0fa24    39           3  Partly cloudy     0.0        56   
3  43877  C-b5282c3b    28          12         Cloudy     0.0        83   
4  16781  C-ff8c0c3c    30           4          Clear     0.0        46   

   visibility  pressure  chanceofrain  chanceoffog  chanceofsnow  \
0           6      1017             0            0             0   
1           6      1028             0     

In [20]:
feature_group_data.keys()

dict_keys(['city_weather_features', 'drivers_features', 'routes_features', 'routes_weather_features', 'traffic_table_features', 'truck_schedule_features', 'trucks_features'])

In [21]:
city_weather = feature_group_data['city_weather_features']
drivers_table = feature_group_data['drivers_features']
routes_table = feature_group_data['routes_features']
routes_weather = feature_group_data['routes_weather_features']
traffic_table = feature_group_data['traffic_table_features'] 
truck_schedule_table = feature_group_data['truck_schedule_features']
trucks_table = feature_group_data['trucks_features']


city_weather.head()

,index,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,datetime,event_date
0,33470,C-2c9e75ef,48,7,Cloudy,0.0,82,6,1017,0,0,0,0,2019-01-16 14:00:00+00:00,2024-10-09 00:00:00+00:00
1,52207,C-841ebdcb,48,7,Sunny,0.0,78,6,1028,0,0,0,0,2019-01-15 07:00:00+00:00,2024-10-09 00:00:00+00:00
2,3667,C-4fe0fa24,39,3,Partly cloudy,0.0,56,6,1015,0,0,0,0,2019-01-15 19:00:00+00:00,2024-10-09 00:00:00+00:00
3,43877,C-b5282c3b,28,12,Cloudy,0.0,83,6,1020,0,0,0,0,2019-02-05 05:00:00+00:00,2024-10-09 00:00:00+00:00
4,16781,C-ff8c0c3c,30,4,Clear,0.0,46,6,1031,0,0,0,0,2019-01-10 05:00:00+00:00,2024-10-09 00:00:00+00:00


In [22]:
routes_weather.head()

,index,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder,event_date
0,155203,R-bf33b15f,2019-01-25 00:00:00,55,5,Clear,0.0,70,6,1014,0,0,0,0,2024-10-09 00:00:00+00:00
1,139437,R-ccaa39f5,2019-01-13 06:00:00,75,8,Sunny,0.0,40,6,1019,0,0,0,0,2024-10-09 00:00:00+00:00
2,77328,R-dddc4e43,2019-02-04 06:00:00,68,6,Patchy rain possible,0.0,96,6,1012,0,0,0,0,2024-10-09 00:00:00+00:00
3,327869,R-27cc4f57,2019-02-08 18:00:00,66,20,Light rain shower,0.0,71,6,1005,0,0,0,0,2024-10-09 00:00:00+00:00
4,284980,R-af3e42c5,2019-01-15 12:00:00,84,13,Sunny,0.0,18,6,1014,0,0,0,0,2024-10-09 00:00:00+00:00


In [26]:
def assign_feature_group_data(feature_group_data, table_names):
    # Print available keys in feature_group_data for debugging
    print("Available keys in feature_group_data:", list(feature_group_data.keys()))
    
    # Dynamically assign the DataFrame from feature_group_data to the variable names provided in table_names
    for table_name in table_names:
        # Append '_features' to match the keys in feature_group_data
        features_key = table_name + '_features'
        if features_key in feature_group_data:
            globals()[table_name] = feature_group_data[features_key]
            print(f"Assigned {table_name} to variable.")
        else:
            print(f"KeyError: '{features_key}' not found in feature_group_data.")

# List of table names without '_features' suffix
table_names = ['city_weather', 'routes_weather', 'traffic_table', 'truck_schedule', 'trucks']

# Call the function to assign feature group data to variables
assign_feature_group_data(feature_group_data, table_names)

# Now you can access the variables dynamically assigned
print(truck_schedule.head())


Available keys in feature_group_data: ['city_weather_features', 'drivers_features', 'routes_features', 'routes_weather_features', 'traffic_table_features', 'truck_schedule_features', 'trucks_features']
Assigned city_weather to variable.
Assigned routes_weather to variable.
Assigned traffic_table to variable.
Assigned truck_schedule to variable.
Assigned trucks to variable.
   index  truck_id    route_id       departure_date  \
0   8028  18507407  R-4b2be088  2019-02-09 07:00:00   
1   8522  25800303  R-8e665007  2019-02-09 07:00:00   
2   3667  56029484  R-144b0d88  2019-01-16 07:00:00   
3   6204  19486564  R-6b21c069  2019-01-31 07:00:00   
4    575  11972697  R-c3bd7d11  2019-01-01 07:00:00   

               estimated_arrival  delay                event_date  
0  2019-02-09 20:49:48.000000000      0 2024-10-09 00:00:00+00:00  
1  2019-02-09 12:24:00.000000000      0 2024-10-09 00:00:00+00:00  
2  2019-01-16 11:48:00.000000000      1 2024-10-09 00:00:00+00:00  
3  2019-01-31 11:43:1

In [28]:

dfs = {
    'city_weather': city_weather,
    'drivers_table': drivers_table,
    'routes_table': routes_table,
    'routes_weather': routes_weather,
    'traffic_table': traffic_table,
    'truck_schedule_table': truck_schedule_table,
    'trucks_table': trucks_table
}


def drop_columns_from_tables(dfs, columns_to_drop):
    """
    Drops specified columns from all DataFrames in the provided dictionary.
    
    Parameters:
    - dfs (dict): A dictionary where the keys are DataFrame names and the values are pandas DataFrames.
    - columns_to_drop (list): A list of column names to be dropped from each DataFrame.
    
    Returns:
    - dict: A dictionary of DataFrames with the specified columns removed.
    """
    
    # Loop through each DataFrame in the dictionary
    for table_name, df in dfs.items():
        # Check if the columns exist in the DataFrame before dropping
        cols_to_drop = [col for col in columns_to_drop if col in df.columns]
        
        # If there are columns to drop, drop them
        if cols_to_drop:
            dfs[table_name] = df.drop(columns=cols_to_drop, inplace=True)
            print(f"Dropped columns {cols_to_drop} from {table_name}.")
        else:
            print(f"No matching columns to drop in {table_name}.")
    
    return dfs

# Example usage:
columns_to_drop = ['index', 'event_date']  # Example columns that might be present in multiple tables
feature_group_data_cleaned = drop_columns_from_tables(feature_group_data, columns_to_drop)




Dropped columns ['index', 'event_date'] from city_weather_features.
Dropped columns ['index', 'event_date'] from drivers_features.
Dropped columns ['index', 'event_date'] from routes_features.
Dropped columns ['index', 'event_date'] from routes_weather_features.
Dropped columns ['index', 'event_date'] from traffic_table_features.
Dropped columns ['index', 'event_date'] from truck_schedule_features.
Dropped columns ['index', 'event_date'] from trucks_features.


In [30]:
def drop_duplicates_from_dfs(dfs, columns_to_check):   
    for df_name, df in dfs.items():
        if df_name in columns_to_check:
            subset_columns = columns_to_check[df_name]
            
            # Check if all subset columns exist in the DataFrame
            missing_columns = [col for col in subset_columns if col not in df.columns]
            if missing_columns:
                print(f"Skipping {df_name}: Missing columns {missing_columns}")
                continue

            before_dropping = df.shape[0]  # Count the number of rows before dropping duplicates
            df.drop_duplicates(subset=subset_columns, inplace=True)
            after_dropping = df.shape[0]  # Count the number of rows after dropping duplicates
            print(f"Dropped {before_dropping - after_dropping} duplicate rows from {df_name} based on columns: {subset_columns}")
        else:
            print(f"No duplicate check columns provided for {df_name}")

# Dictionary where the key is the DataFrame name, and the value is the subset of columns to check for duplicates
columns_to_check = {
    'city_weather': ['city_id', 'date'],
    'route_weather': ['route_id', 'date'],
    'trucks_table': ['truck_id'],
    'drivers_table': ['driver_id'],
    'routes_table': ['route_id', 'destination_id', 'origin_id'],
    'truck_schedule': ['truck_id', 'route_id', 'departure_date']
}

# Call the function to drop duplicates from the DataFrames
drop_duplicates_from_dfs(dfs, columns_to_check)

# Check the result by printing the modified DataFrames
for df_name, df in dfs.items():
    print(f"\n{df_name} DataFrame after dropping duplicates:")
    print(df)


Skipping city_weather: Missing columns ['date']
Dropped 0 duplicate rows from drivers_table based on columns: ['driver_id']
Dropped 0 duplicate rows from routes_table based on columns: ['route_id', 'destination_id', 'origin_id']
No duplicate check columns provided for routes_weather
No duplicate check columns provided for traffic_table
No duplicate check columns provided for truck_schedule_table
Dropped 0 duplicate rows from trucks_table based on columns: ['truck_id']

city_weather DataFrame after dropping duplicates:
          city_id  temp  wind_speed        description  precip  humidity  \
0      C-2c9e75ef    48           7             Cloudy     0.0        82   
1      C-841ebdcb    48           7              Sunny     0.0        78   
2      C-4fe0fa24    39           3      Partly cloudy     0.0        56   
3      C-b5282c3b    28          12             Cloudy     0.0        83   
4      C-ff8c0c3c    30           4              Clear     0.0        46   
...           ...   

In [38]:
#code to drop the unnecessary columns in the city_weather dataframe
# List of columns to drop
columns_to_drop = ['chanceofrain', 'chanceoffog', 'chanceofsnow', 'chanceofthunder']

# Drop the specified columns from the city_weather DataFrame
city_weather.drop(columns=columns_to_drop, inplace=True)
routes_weather.drop(columns=columns_to_drop, inplace=True)

# Display the updated DataFrame
city_weather.head()
routes_weather.head()

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-553cacdb,2019-01-18 18:00:00,64.0,19.0,Light rain shower,0.0,94.0,6.0,1006.0
1,R-d60a0eef,2019-01-13 12:00:00,81.0,9.0,Sunny,0.0,74.0,6.0,1015.0
2,R-e7461db1,2019-02-08 12:00:00,43.0,11.0,Clear,0.0,78.0,6.0,1008.0
3,R-ea0491d1,2019-02-09 00:00:00,66.0,7.0,Sunny,0.0,67.0,6.0,1022.0
4,R-e2293958,2019-02-02 12:00:00,48.0,2.0,Clear,0.0,59.0,6.0,1021.0


In [39]:
# Rename the 'date' column to 'custom_date'
city_weather.rename(columns={'date': 'custom_date'}, inplace=True)

# Display the updated DataFrame
city_weather.head()

,city_id,custom_date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure
0,C-19236709,2019-01-11,1200.0,32.0,6.0,Cloudy,0.0,52.0,6.0,1026.0
1,C-2e349ccd,2019-02-03,1800.0,81.0,14.0,Clear,0.0,23.0,6.0,1012.0
2,C-2bd47dc5,2019-01-01,1900.0,41.0,11.0,Clear,0.0,59.0,6.0,1020.0
3,C-b5ff31cd,2019-01-11,1400.0,19.0,15.0,Sunny,0.0,36.0,6.0,1022.0
4,C-b6e04c88,2019-01-30,600.0,1.0,16.0,Overcast,0.0,47.0,6.0,1029.0


In [49]:
city_weather.head()

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,C-19236709,2019-01-11,1200.0,32.0,6.0,Cloudy,0.0,52.0,6.0,1026.0,0.0,0.0,0.0,0.0
1,C-2e349ccd,2019-02-03,1800.0,81.0,14.0,Clear,0.0,23.0,6.0,1012.0,0.0,0.0,0.0,0.0
2,C-2bd47dc5,2019-01-01,1900.0,41.0,11.0,Clear,0.0,59.0,6.0,1020.0,0.0,0.0,0.0,0.0
3,C-b5ff31cd,2019-01-11,1400.0,19.0,15.0,Sunny,0.0,36.0,6.0,1022.0,0.0,0.0,0.0,0.0
4,C-b6e04c88,2019-01-30,600.0,1.0,16.0,Overcast,0.0,47.0,6.0,1029.0,0.0,0.0,0.0,0.0


In [51]:
traffic_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459682 entries, 0 to 2459681
Data columns (total 5 columns):
 #   Column          Dtype  
---  ------          -----  
 0   route_id        object 
 1   date            object 
 2   hour            float64
 3   no_of_vehicles  float64
 4   accident        float64
dtypes: float64(3), object(2)
memory usage: 93.8+ MB


In [58]:
traffic_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459682 entries, 0 to 2459681
Data columns (total 5 columns):
 #   Column          Dtype  
---  ------          -----  
 0   route_id        object 
 1   date            object 
 2   hour            float64
 3   no_of_vehicles  float64
 4   accident        float64
dtypes: float64(3), object(2)
memory usage: 93.8+ MB
